In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1733078893450_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1733078893450_0001,pyspark,idle,Link,Link,None,✔


In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
PATH = 's3://albucket-p8-opc/train_image/'
PATH_Data = "s3://albucket-p8-opc/train_image/Test"
PATH_Result = PATH+'Results'

# Affichage des chemins pour vérification
print('PATH:        ' + PATH +
      '\nPATH_Data:   ' + PATH_Data +
      '\nPATH_Result: ' + PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://albucket-p8-opc/train_image/
PATH_Data:   s3://albucket-p8-opc/train_image/Test
PATH_Result: s3://albucket-p8-opc/train_image/Results

In [5]:
all_files = spark.read.format("binaryFile").load("s3://albucket-p8-opc/train_image/apple_6/")
all_files.select("path").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------+
|path                                               |
+---------------------------------------------------+
|s3://albucket-p8-opc/train_image/apple_6/r1_288.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_284.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_286.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_276.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_308.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_274.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_282.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_280.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_296.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_298.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_292.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_310.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_290.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_294.jpg|
|s3://albucket-p8-opc/train_image/apple_6/r1_306.jpg|
|s3://albucket-p8-opc/train_

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load("s3://albucket-p8-opc/train_image/")
images.show(1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://albucket-p8-...|2024-11-12 00:21:01|125373|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 1 row

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://albucket-p8-...|2024-11-12 00:21:01|125373|[FF D8 FF E0 00 1...|
|s3://albucket-p8-...|2024-11-12 00:20:59|125088|[FF D8 FF E0 00 1...|
|s3://albucket-p8-...|2024-11-12 00:20:59|124905|[FF D8 FF E0 00 1...|
|s3://albucket-p8-...|2024-11-12 00:21:01|124363|[FF D8 FF E0 00 1...|
|s3://albucket-p8-...|2024-11-12 00:21:02|124026|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------------------+-----------+
|path                                                   |label      |
+-------------------------------------------------------+-----------+
|s3://albucket-p8-opc/train_image/apple_hit_1/r0_116.jpg|apple_hit_1|
|s3://albucket-p8-opc/train_image/apple_hit_1/r0_114.jpg|apple_hit_1|
|s3://albucket-p8-opc/train_image/apple_hit_1/r0_108.jpg|apple_hit_1|
|s3://albucket-p8-opc/train_image/apple_hit_1/r0_118.jpg|apple_hit_1|
|s3://albucket-p8-opc/train_image/apple_hit_1/r0_120.jpg|apple_hit_1|
+-------------------------------------------------------+-----------+
only showing top 5 rows

None

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "functional"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

from typing import Iterator

@pandas_udf('array<float>')
def featurize_udf(content_series_iter: Iterator[pd.Series]) -> Iterator[pd.Series]:
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # Load the model once and reuse it
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://albucket-p8-opc/train_image/Results

In [18]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Ajout d’une PCA de Réduction de Dimensions

In [19]:
features_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------------------+
|                path|          label|            features|
+--------------------+---------------+--------------------+
|s3://albucket-p8-...|    apple_hit_1|[0.088961504, 0.5...|
|s3://albucket-p8-...|    apple_hit_1|[0.021604996, 0.2...|
|s3://albucket-p8-...|    apple_hit_1|[0.32612726, 0.36...|
|s3://albucket-p8-...|    apple_hit_1|[0.8480136, 0.371...|
|s3://albucket-p8-...|    apple_hit_1|[0.8983205, 0.488...|
|s3://albucket-p8-...|    apple_hit_1|[0.6280044, 0.241...|
|s3://albucket-p8-...|    apple_hit_1|[0.029147508, 0.1...|
|s3://albucket-p8-...|    apple_hit_1|[0.09389346, 0.25...|
|s3://albucket-p8-...|    apple_hit_1|[0.21333878, 0.52...|
|s3://albucket-p8-...|cabbage_white_1|[0.0, 0.92801636,...|
|s3://albucket-p8-...|cabbage_white_1|[0.0, 0.5157436, ...|
|s3://albucket-p8-...|    apple_hit_1|[1.2991298, 0.004...|
|s3://albucket-p8-...|    apple_hit_1|[1.3367201, 0.366...|
|s3://albucket-p8-...|    apple_hit_1|[2

In [20]:
# Sélectionner uniquement les colonnes 'label' et 'features'
features_selected_df = features_df.select("label", "features")

# Afficher les premières lignes pour vérifier le résultat
features_selected_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+
|          label|            features|
+---------------+--------------------+
|    apple_hit_1|[0.021604996, 0.2...|
|    apple_hit_1|[0.36675403, 0.42...|
|    apple_hit_1|[0.37229705, 0.25...|
|    apple_hit_1|[0.2765281, 0.636...|
|    apple_hit_1|[0.01139562, 0.50...|
|    apple_hit_1|[0.054030925, 0.3...|
|    apple_hit_1|[0.113923736, 0.1...|
|    apple_hit_1|[0.006236485, 0.2...|
|    apple_hit_1|[0.17542247, 0.50...|
|cabbage_white_1|[0.0, 0.6001656, ...|
|cabbage_white_1|[0.0, 1.7821127, ...|
|cabbage_white_1|[0.0, 1.7985052, ...|
|    apple_hit_1|[0.5980094, 0.126...|
|cabbage_white_1|[0.0, 0.4861757, ...|
|cabbage_white_1|[0.0, 0.36072424,...|
|cabbage_white_1|[0.0, 1.1773732, ...|
|cabbage_white_1|[0.0, 0.46463066,...|
|    apple_hit_1|[1.1884638, 0.674...|
|    apple_hit_1|[1.032343, 0.0916...|
|    apple_hit_1|[0.42595816, 0.54...|
+---------------+--------------------+
only showing top 20 rows

In [21]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.linalg import DenseVector

# Convertir la colonne 'features' en DenseVector
def array_to_vector(array):
    return DenseVector(array)

# UDF pour la conversion
# Use VectorUDT as the returnType instead of DenseVector
array_to_vector_udf = udf(array_to_vector, VectorUDT())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Appliquer la conversion au DataFrame
features_vector_df = features_selected_df.withColumn("features_vector", array_to_vector_udf("features"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
features_vector_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+--------------------+
|          label|            features|     features_vector|
+---------------+--------------------+--------------------+
|    apple_hit_1|[0.021604996, 0.2...|[0.02160499617457...|
|    apple_hit_1|[0.36675403, 0.42...|[0.36675402522087...|
|    apple_hit_1|[0.37229705, 0.25...|[0.37229704856872...|
|    apple_hit_1|[0.2765281, 0.636...|[0.27652809023857...|
|    apple_hit_1|[0.01139562, 0.50...|[0.01139562018215...|
|    apple_hit_1|[0.054030925, 0.3...|[0.05403092503547...|
|    apple_hit_1|[0.113923736, 0.1...|[0.11392373591661...|
|    apple_hit_1|[0.006236485, 0.2...|[0.00623648520559...|
|    apple_hit_1|[0.17542247, 0.50...|[0.17542247474193...|
|cabbage_white_1|[0.0, 0.6001656, ...|[0.0,0.6001656055...|
|cabbage_white_1|[0.0, 1.7821127, ...|[0.0,1.7821127176...|
|cabbage_white_1|[0.0, 1.7985052, ...|[0.0,1.7985051870...|
|    apple_hit_1|[0.5980094, 0.126...|[0.59800940752029...|
|cabbage_white_1|[0.0, 0.4861757, ...|[0

In [24]:
from pyspark.ml.feature import PCA

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Configurer la PCA
num_components = 200
pca = PCA(k=num_components, inputCol="features_vector", outputCol="pca_features")

# Appliquer la PCA
pca_model = pca.fit(features_vector_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
print(pca.explainParams())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

inputCol: input column name. (current: features_vector)
k: the number of principal components (current: 200)
outputCol: output column name. (default: PCA_fa62fc9af27d__output, current: pca_features)

In [27]:
# Afficher les informations du modèle PCA
print("Matrice des composantes principales (PC):")
print(pca_model.pc.toArray())  # Affiche les composantes principales sous forme de matrice
print( )

# Afficher la variance expliquée 
print("Variance expliquée par chaque composante :")
print(pca_model.explainedVariance)
print( )

print(f"\nNombre de composantes principales selectionnees : {pca.getK()}")
print(f"Colonne d'entrée : {pca.getInputCol()}")
print(f"Colonne de sortie : {pca.getOutputCol()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matrice des composantes principales (PC):
[[-0.00669023  0.00089857 -0.02156362 ...  0.03608194  0.00441987
   0.004575  ]
 [ 0.01089797  0.02781554 -0.03582205 ...  0.05432643  0.02502721
   0.04396769]
 [ 0.02357434 -0.01816372  0.01860074 ...  0.01236417  0.01655389
   0.01477603]
 ...
 [ 0.00103085  0.00050652 -0.00135887 ...  0.00497867  0.00439721
   0.00381603]
 [ 0.02250618 -0.02788043  0.01530427 ... -0.06437043 -0.02809145
   0.02038383]
 [ 0.01372698 -0.03443701 -0.02730697 ...  0.00402036 -0.02540994
  -0.00805341]]

Variance expliqu?e par chaque composante :
[0.14522913724033637,0.11517044880457901,0.07371454121104151,0.050727207582734954,0.03899935164000868,0.03531231806956625,0.029265532959878608,0.026460279796506486,0.022657446857571595,0.02161533292899548,0.01935781696458293,0.017126856427859143,0.01600811224119546,0.014041770974502233,0.012214026459872296,0.011260583518586666,0.010749364007264565,0.010153151238208407,0.00959458632358842,0.008980921964588447,0.00834725

In [28]:
# Calculer la variance expliquée pour chaque composante
explained_variance = pca_model.explainedVariance

# Calculer la variance totale expliquée par les 200 premières composantes
total_explained_variance = sum(explained_variance[:200])

print(f"La variance expliquee par les 200 premieres composantes est : {total_explained_variance:.4f}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

La variance expliquee par les 200 premieres composantes est : 0.9445

In [29]:
import matplotlib.pyplot as plt

# Extraire la variance expliquée par chaque composante
explained_variance_array = np.array(explained_variance)

# Calculer la variance expliquée cumulée
cumulative_variance = np.cumsum(explained_variance_array)

# Tracer le graphique
plt.figure(figsize=(12, 6))

# Tracé de la variance expliquée pour chaque composante
plt.bar(range(1, len(explained_variance_array) + 1), explained_variance_array, alpha=0.6, label='Variance expliquée par composante')

# Tracé de la variance cumulative
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', color='red', label='Variance expliquée cumulative')

# Ajouter une ligne pour la variance totale expliquée par les 200 premières composantes
plt.axhline(y=total_explained_variance, color='green', linestyle='--', label=f'Variance totale expliquée (200 premières composantes): {total_explained_variance:.3f}')

# Titres et légendes
plt.xlabel("Nombre de composantes principales")
plt.ylabel("Variance expliquée")
plt.title("Variance expliquée par chaque composante principale et variance cumulative")
plt.legend()
plt.grid(True)
plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Afficher les résultats
pca_result_df = pca_model.transform(features_vector_df).select("label", "pca_features")
pca_result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+
|          label|        pca_features|
+---------------+--------------------+
|    apple_hit_1|[7.31946079895336...|
|    apple_hit_1|[5.80980287813219...|
|    apple_hit_1|[3.07168712263963...|
|    apple_hit_1|[4.20386081805463...|
|    apple_hit_1|[5.91996610459940...|
|    apple_hit_1|[4.93838800947562...|
|    apple_hit_1|[5.25003964732713...|
|    apple_hit_1|[5.34376369742085...|
|    apple_hit_1|[6.33651004643266...|
|cabbage_white_1|[5.19048856100703...|
|cabbage_white_1|[6.79330404563015...|
|cabbage_white_1|[6.65893070387620...|
|    apple_hit_1|[2.03665103517866...|
|cabbage_white_1|[6.86615956712201...|
|cabbage_white_1|[5.87164923059506...|
|cabbage_white_1|[5.39496702463002...|
|cabbage_white_1|[5.94277739759814...|
|    apple_hit_1|[5.84126947296662...|
|    apple_hit_1|[4.25223812933776...|
|    apple_hit_1|[4.99099125468725...|
+---------------+--------------------+
only showing top 20 rows

In [31]:
# Convertir le DataFrame Spark en Pandas DataFrame pour l'affichage
pca_result_pd = pca_result_df.select("label", "pca_features").toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Plot 2D

In [32]:
# Extraire uniquement les deux premières composantes de chaque vecteur de `pca_features`
pca_result_pd[['pca1', 'pca2']] = pd.DataFrame(pca_result_pd['pca_features'].apply(lambda x: x[:2]).tolist(), index=pca_result_pd.index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Plot en 2D
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 7))
for label in pca_result_pd['label'].unique():
    subset = pca_result_pd[pca_result_pd['label'] == label]
    plt.scatter(subset['pca1'], subset['pca2'], label=label, alpha=0.5)

plt.xlabel('Première composante principale')
plt.ylabel('Deuxième composante principale')
plt.title('Projection PCA (deux premières composantes)')
plt.legend()
plt.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Plot 3D

In [34]:
#Convertir le DataFrame Spark en Pandas DataFrame pour l'affichage
pca_result_pd = pca_result_df.select("label", "pca_features").toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# Extraire uniquement les trois premières composantes de chaque vecteur de `pca_features`
pca_features_list = pca_result_pd['pca_features'].apply(lambda x: x[:3]).tolist()
pca_result_pd[['pca1', 'pca2', 'pca3']] = pd.DataFrame(pca_features_list, index=pca_result_pd.index)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
from mpl_toolkits.mplot3d import Axes3D

# Création du graphique en 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Boucle sur chaque label unique pour les afficher avec des couleurs différentes
for label in pca_result_pd['label'].unique():
    subset = pca_result_pd[pca_result_pd['label'] == label]
    ax.scatter(subset['pca1'], subset['pca2'], subset['pca3'], label=label, alpha=0.6)

# Labels des axes
ax.set_xlabel('Première composante principale')
ax.set_ylabel('Deuxième composante principale')
ax.set_zlabel('Troisième composante principale')
ax.set_title('Projection PCA (trois premières composantes)')
ax.legend()
plt.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
# Sauvegarder les résultats en parquet
pca_result_df.write.mode("overwrite").parquet(PATH_Result + "/pca_result")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Chargement des données enregistrées et validation du résultat

In [38]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0  s3://albucket-p8-opc/train_image/apple_hit_1/r...  ...  [0.088961504, 0.5938357, 1.22061, 0.0, 1.13357...
1  s3://albucket-p8-opc/train_image/apple_hit_1/r...  ...  [0.021604996, 0.27189785, 0.46618503, 0.0, 0.4...
2  s3://albucket-p8-opc/train_image/apple_hit_1/r...  ...  [0.32612726, 0.36732003, 1.0039239, 0.0, 0.966...
3  s3://albucket-p8-opc/train_image/apple_hit_1/r...  ...  [0.8480136, 0.37196302, 2.2546833, 0.0, 0.5414...
4  s3://albucket-p8-opc/train_image/apple_hit_1/r...  ...  [0.8983205, 0.48892242, 1.1914114, 0.0, 0.3367...

[5 rows x 3 columns]

In [40]:
df.loc[0,'features'].shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(1280,)

In [41]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(12462, 3)